## Machine Learning With sci-kit learn: Regression

Scikit-learn is a cornerstone in the Python machine learning toolkit, especially when dealing with tabular data. It's the go-to for scenarios where we're not tackling unstructured data types like text or images but focusing on more classic, structured datasets.

The library's compatibility with Pandas and Numpy is seamless, forming a robust stack for data manipulation and analysis. Scikit-learn's API is designed for consistency and ease of use, which streamlines the process of building and deploying machine learning models. It's not just about model creation; the library also provides essential tools for model evaluation and selection, ensuring we can measure and improve our approach systematically.

With scikit-learn, you have access to a broad array of algorithms for various machine learning tasks—classification, regression, clustering, and more. It's also equipped with features for preprocessing data, selecting the right features, and fine-tuning models through cross-validation and hyperparameter optimization.

## Importing packages and data

In [1]:
# %pip install -U scikit-learn 

In [2]:
import pandas as pd 
import numpy  as np
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [3]:
pd.options.display.float_format = '{:.2f}'.format


In [4]:
travel_dataset = pd.read_csv("data/output/travel_dataset_cleaned.csv")

In [5]:
travel_dataset.head()

,sales_id,age,country,membership_status,previous_purchases,destination,stay_length,guests,travel_month,months_before_travel,earlybird_discount,package_Type,cost,rating,margin,additional_services_cost
0,1,47,Italy,standard,3,Tokyo,8,3,2,2,False,Relaxation,2899.26,6,1865.22,527.18
1,2,37,Canada,standard,6,Cairo,4,2,8,1,False,Cultural,986.72,5,880.26,167.86
2,3,49,Australia,silver,3,Paris,4,2,12,1,True,Adventure,1610.70,6,914.26,298.25
3,4,62,Germany,gold,7,Paris,8,1,12,11,True,Adventure,2155.61,8,1512.81,415.49
4,5,36,UK,gold,4,Rio,5,3,8,4,False,Relaxation,1183.84,7,839.05,200.58


## Useful functions 

In [6]:
def plot_results(y_true: pd.Series, predictions: np.ndarray, title: str) -> None:
    
    fig = px.scatter(x=predictions, y=y_true, labels={"x": "predicted", "y": "actual"}, title=title)
    fig.add_shape(type="line",
                x0=-1000, 
                y0=-1000, 
                x1=7000, 
                y1=7000)
    
    fig.show()

## predict the cost of a given stay

**In this case study we want to predict the cost of a stay.**

### Step 1: Splitting data in training and test set 

In [7]:
X = travel_dataset.drop(columns="cost") # by convention the variables are called (capital) X
y = travel_dataset["cost"] # By convention what you want to predict is called (small letter) y

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [9]:
print(f"{len(X_train) = } and {len(X_test) = }")

len(X_train) = 40000 and len(X_test) = 10000


### Step 2: Preprocessing

Some important steps preceding training a machine learning model: 
- One Hot Encoding 
- Scaling 
- Creating polynominal features (interaction terms)
- Binning
- Dealing with outliers and missing values 

Depending on the data type, different preprocessing steps are needed

In scikit-learn, the Pipeline and ColumnTransformer classes offer an idiomatic and streamlined way to chain multiple preprocessing steps and a model into a single workflow. The `Pipeline` class allows you to assemble sequences of transformations and a final model, which simplifies your code and helps prevent common mistakes, such as fitting preprocessing steps to the test data. The `ColumnTransformer` is particularly useful for applying different preprocessing to different columns, such as one-hot encoding for categorical variables and scaling for numerical variables. By combining these tools, you not only reduce the need for manual 'glue' code but also safeguard against the leakage of information from the test set into the training process. We highly recommend you always use this in the scope of this course.

The syntax is quite simple:

*  `make_column_transformer` expects multiple tuples. The `transformer` (preprocessing step) is in the first position of the tuple and the columns you apply the transformer (type: `list[str]`) is in the second position.  
*  `make_pipeline` similarly expects all objects in sequence, so typically you add your preprocessing first followed by the model you want to apply. 


In [10]:
cat_columns = ["package_Type", "destination", "country"]
numeric_columns = ["guests", "age", "stay_length"]
bin_columns = ["cost"]


In [11]:
poly = PolynomialFeatures(interaction_only=True)
numeric_preprocessing = make_pipeline(StandardScaler(), poly)

In [12]:
preprocessing = make_column_transformer(
    (numeric_preprocessing, numeric_columns),
    (OneHotEncoder(sparse_output=False),cat_columns),
    (KBinsDiscretizer(),["age"]),
    remainder="drop"
)

### Step 3: Model training + Model prediction

#### Linear Regression

In [13]:
lin_reg_pipe = make_pipeline(preprocessing, LinearRegression())

In [14]:
lin_reg_pipe.fit(X_train, y_train)
predictions_lin_reg_train = lin_reg_pipe.predict(X_train)
predictions_lin_reg_test = lin_reg_pipe.predict(X_test)

In [15]:
plot_results(y_train, predictions_lin_reg_train, title="predicted vs actual for linear regression on the training set")

In [16]:
plot_results(y_test, predictions_lin_reg_test, title="predicted versus actual on the test set for linear regression")

#### Decision Tree 

In [17]:
dt_reg_pipe = make_pipeline(preprocessing, DecisionTreeRegressor())

In [18]:
dt_reg_pipe.fit(X_train, y_train)
predictions_dt_reg_train = lin_reg_pipe.predict(X_train)
predictions_dt_reg_test = lin_reg_pipe.predict(X_test)

In [19]:
plot_results(y_train, predictions_dt_reg_train, title="predicted vs actual for linear regression on the training set")

In [20]:
plot_results(y_test, predictions_dt_reg_test, title="predicted versus actual on the test set for linear regression")

### Step 4: Model Evaluation 

 Compute both the MAE (Mean Average Error) and the RMSE (Root Mean Squared Error):
* The one you should focus on the most is typically the RMSE. 
* If there are error outliers then the difference between the MAE and the RMSE is likely going to be large. In that case it is typically more interesting to look at the MAE. 
 
Typically, RMSE is preferred because it is more sensitive to large errors, which can be critical in applications where such errors are especially problematic, like in autonomous vehicle guidance systems. However, if your model is prone to outliers, or if large errors are less impactful, MAE can be a more relevant metric. Always consider the specific context of your application when choosing your primary evaluation metric

Squaring the errors, as shown in the equation: $(-2)² = 4 = (2)²$, ensures that all error values are positive. This method has two main benefits:

This method has two main advantages:

1. Large errors are amplified more than smaller ones, which can be particularly important in cases where larger deviations are less tolerable.
2. MSE is a differentiable function, which makes it mathematically convenient for optimization algorithms used in model training. There are other statistical properties that the MSE can leverage.

Note that the MSE can be more difficult to interpret since it's not in the same units as the original data. Taking the square root to obtain the RMSE helps mitigate this issue.

Let's train 4 different algoritms:
- Linear Regression 
- Decision Tree Regression 
- Random Forest Regression 
- HistGradientBoosting Regression


In [21]:
model_name_pair = [("random_forest", RandomForestRegressor(n_jobs=-1)), ("gradient boosting", HistGradientBoostingRegressor()), ("decision tree", DecisionTreeRegressor()), ("linear regression", LinearRegression()) ]
results = []
for pair in model_name_pair:
    name, model = pair
    print(f"STARTING {name}")
    pipe = make_pipeline(preprocessing, model)
    pipe.fit(X_train, y_train)
    predictions_train = pipe.predict(X_train)
    predictions_test = pipe.predict(X_test)
    print("-"*20)
    print(f"The MAE on the training set is {mean_absolute_error(y_train, predictions_train)} and on the test set it is {mean_absolute_error(y_test, predictions_test)}")
    print(f"The RMSE on the training set is {root_mean_squared_error(y_train, predictions_train)} and on the test set it is {root_mean_squared_error(y_test, predictions_test)}\n")

STARTING random_forest
--------------------
The MAE on the training set is 91.66538082926094 and on the test set it is 235.292119500531
The RMSE on the training set is 124.67274414988401 and on the test set it is 314.2982681603037

STARTING gradient boosting
--------------------
The MAE on the training set is 214.5016864287823 and on the test set it is 223.33350849067224
The RMSE on the training set is 268.64954699702605 and on the test set it is 285.6096684051547

STARTING decision tree
--------------------
The MAE on the training set is 12.197864719583333 and on the test set it is 281.1946939391666
The RMSE on the training set is 51.045890849136754 and on the test set it is 414.47735341906184

STARTING linear regression
--------------------
The MAE on the training set is 301.14818619749997 and on the test set it is 296.79584943
The RMSE on the training set is 407.3063644157695 and on the test set it is 399.5555293467178



It seems like the decision tree and random forest are both overfitting. There is a large gap between the performance on the test set and the training set.
Linear regression is underfitting, the performance on test and train is similarly bad. 
Gradient boosting is not underfitting nor overfitting. The difference between test and train is minimal, on top of that, its MAE and MSE are better than all the alternatives we have tried.

### Step 5: Finding best model: cross validation

**Cross-validation**

 We divide our training set into smaller sections, say 20% chunks. We train our model on 80% of the data, then validate it on the remaining 20%. We repeat this process five times, each time with a different 20% held out for validation. This technique, known as k-fold cross-validation (with k being the number of chunks or 'folds' we create), allows each model a fair shot at proving itself across the entirety of our data.

By averaging the performance across these folds, we obtain a more reliable measure of a model's quality. This thorough approach increases our confidence that we're selecting the best model, not by chance, but by consistent performance.

!!! Note, since you're training a lot of models it could take a prohibitively long time to complete.

As usual, sci-kit learn makes it easy to do cross validation. 

We supply `cross_val_score` with 3 mandatory parameters:

* The machine learning model
* The `X_train`
* `X_test`
  
Additionally you can use `cv` to specify how many folds and you can pick a `score` parameter, which is the result that will be reported to you as the performance on each fold



Let's apply this to our linear regression and decision tree pipeline

In [22]:
# Let's apply this to our linear regression and desicion tree regression model 

lin_reg_cv_results = cross_val_score(lin_reg_pipe, X_train, y_train, cv=5, scoring= "neg_root_mean_squared_error")


In [23]:
lin_reg_cv_results

array([-400.09387232, -419.75949029, -407.14838458, -410.1339624 ,
       -401.64686798])

In [24]:
np.mean(lin_reg_cv_results)

-407.7565155156651

In [25]:
dt_reg_cv_results = cross_val_score(dt_reg_pipe, X_train, y_train, cv=5, scoring= "neg_root_mean_squared_error")


In [26]:
dt_reg_cv_results


array([-423.93771318, -437.66199217, -426.46198322, -417.58361046,
       -417.20885484])

In [27]:
np.mean(dt_reg_cv_results)

-424.57083077242487

**Conclusion**

Linear Regression performs better than the decision tree. When we evaluated the decision tree on the training set it performed really well. Once we added the test set into the mix it became clear the model was overfitting. By using cross validation we are able to see that linear regression is the better model **without** having to use our test set. 

You can experiment with a variety of machine learning models, set-ups (different pre-processing pipelines) and cross-validation. 
By integrating non-linear elements like interactions and binning, linear regression can better capture intricate associations. In contrast, gradient boosting, as a decision tree algorithm, naturally manages complex non-linearities. Comprising an ensemble of decision trees, each subsequent tree aims to rectify errors made by its predecessors. Decision trees inherently account for binning and interactions, thus diminishing the significance of supplementary non-linearities in the context of gradient boosting versus linear regression comparison.

Another thing we can do is called **hyperparameter tuning**. Hyperparameters are simply the "settings" of the machine learning model. Parameters are what it uses to make predictions. So for linear regression the parameters are the coefficients and for decision trees these are the splits it is making. Most machine learning models have hyperparameters (settings) as well. Typically they are used to decrease the complexity of the model. For a decision tree a hyperparameter is for instance the amount of splits it is allowed to make. For Random Forest and Gradient boosting the amount of trees it uses are a hyperparameter. For linear regression the regularization constant (punishment if the coefficients get large) is also a hyperparameter.**Hyperparameters help us combat overfitting.** Some models are very sensitive to their choice of hyperparameters. With their defaults they typically overfit, this is the case for random forest and decision trees (see above). To identify the best hyperparameters, techniques like grid search and random search are used. Grid search exhaustively explores all possible combinations of hyperparameters, offering a thorough but computationally intensive approach. Random search, on the other hand, samples a predetermined number of combinations randomly, providing a quicker but slightly less focused alternative. For each hyperparameter combination we can do K-fold cross validation. This means if we have 2 parameters with each 10 outcomes we have 100 combinations which each need K-1 models, assuming you are doing grid search. This is a very expensive procedure.

The overall process: 

1. You start by dividing your dataset into two parts: training and test data.
2. The training data is then used for cross-validation. This technique is a reliable way to assess model performance and can be paired with hyperparameter tuning, although that's not mandatory.
3. Choose the model or models that perform best in the cross-validation phase.
4. These top models are then trained with the entire set of training data.
5. finally, we test these models on the test data. The results from this step give us our final evaluation metrics.




